Entrenamiento

In [ ]:
# ===============================================
# Setup
# ===============================================
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# Paths
DATA_DIR = "/content/dataset_by_age"

BATCH_SIZE = 16
IMG_SIZE = (128, 128)
NUM_CLASSES = 3

# ===============================================
# Load datasets
# ===============================================
train_ds = image_dataset_from_directory(
    f"{DATA_DIR}/train",
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42
)

val_ds = image_dataset_from_directory(
    f"{DATA_DIR}/validation",
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

test_ds = image_dataset_from_directory(
    f"{DATA_DIR}/test",
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

# ===============================================
# Prefetch for performance
# ===============================================
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

# ===============================================
# Build model with ResNet-50
# ===============================================
base_model = ResNet50(
    include_top=False,
    input_shape=IMG_SIZE + (3,),
    weights='imagenet'
)
base_model.trainable = False

# Add classification head
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ===============================================
# Train
# ===============================================
EPOCHS = 30

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

# ===============================================
# Evaluate on test set
# ===============================================
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc*100:.2f}%")

# ===============================================
# Plot training history (0 to 1.0)
# ===============================================
plt.figure(figsize=(12,5))

# Accuracy plot
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.title('Accuracy')
plt.ylim(0, 1.0)  # set y-axis from 0 to 1
plt.legend()

# Loss plot
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.ylim(0, 1.0)  # set y-axis from 0 to 1
plt.legend()

plt.show()

Guardar el modelo entrenado.

In [ ]:
# Guardar el modelo entrenado como HDF5
model.save("model_age_classification.keras")

In [ ]:
# ===============================================
# 8️⃣ Fine-tuning
# ===============================================

# Unfreeze last layers of ResNet-50
base_model.trainable = True
for layer in base_model.layers[:-50]:  # freeze early layers
    layer.trainable = False

# Re-compile with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune for a few epochs
EPOCHS_FINETUNE = 5
history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINETUNE
)

# ===============================================
# 9️⃣ Plot fine-tuning history (0 to 1)
# ===============================================
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history_finetune.history['accuracy'], label='train acc')
plt.plot(history_finetune.history['val_accuracy'], label='val acc')
plt.title('Fine-tuning Accuracy')
plt.ylim(0, 1.0)
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history_finetune.history['loss'], label='train loss')
plt.plot(history_finetune.history['val_loss'], label='val loss')
plt.title('Fine-tuning Loss')
plt.ylim(0, 1.0)
plt.legend()

plt.show()